In [1]:
import pandahouse as ph
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [2]:
connection_default = {'host': 'http://clickhouse.beslan.pro:8080',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

In [3]:
connection_test = dict(database='test',
                  host='http://clickhouse.beslan.pro:8080',
                  user='student-rw',
                  password='656e2b0c9c')

In [4]:
# Task_1
q = '''
    select 
        count(distinct(st_id)) as hard_students
    from default.peas
    where correct == '1'
    having count(correct) >= 20
    '''
# отправляем запрос и записываем результат в пандасовский датафрейм
task_1 = ph.read_clickhouse(query=q, connection=connection_default)
task_1

,hard_students
0,305


In [5]:
# Task_2
q = '''
    WITH
    (select count(st_id) from default.peas where correct == '1' having count(correct) > '10') as active_users,
    (select distinct count(st_id) from default.peas) as potential,
    (select count(st_id) from (select st_id, count(correct) as tasks from default.peas  where subject == 'Math'  group by st_id 
    having tasks >= '2')) as active_users_math
SELECT
    test_grp,
    (money / st_id_unique) as ARPU,
    (money / active_users) as ARPAU,
    (st_id / potential) * 100 as CR,
    (st_id / active_users) * 100 as CR_active
FROM (
select
    count(B.st_id) as st_id,
    count(DISTINCT B.st_id) as st_id_unique,
    sum(B.money) as money,
    A.test_grp as test_grp
    from
        default.final_project_check as B
    inner join
        default.studs as A
    on 
        A.st_id = B.st_id
group by test_grp)
    '''

q_test = ph.read_clickhouse(query=q, connection=connection_default)
q_test

,test_grp,ARPU,ARPAU,CR,CR_active
0,control,92333.333333,204.760497,0.135900,0.266115
1,pilot,106093.750000,501.921940,0.324651,0.635719
